In [ ]:
### TODO Last Updated - 11.09.25 1231Z 

### TODO, short term (by 11.16)

### TODO, medium term (by 11.25)
### Develop test using Codex that makes sure that calculations are correct

### TODO, long term (12.09)
### Develop method for testing NAM3K against RTMA
### Develop method for testing HRRR against NAM3K
### Develop method for testing HRRR/NAM3K against URMA

from herbie import Herbie
from herbie.toolbox import EasyMap, pc
from herbie import paint
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta

In [ ]:
date = input("Enter date (YYYY-MM-DD): ")
init_hour = int(input("Enter initialization hour (HH): "))
forecast = int(input("Enter forecast hour (0..N): "))

# Build the HRRR cycle datetime (initialization time)
cycle_dt = datetime.fromisoformat(f"{date} {init_hour:02d}:00")

# Compute the forecast valid time (handles day/month/year rollovers)
valid_dt = cycle_dt + timedelta(hours=forecast)

# Optional: a display string if you need it
displayhour = valid_dt.strftime("%H:%M")

# Compute display hour safely
displayhour = (forecast + init_hour) % 24

# HRRR: pass the cycle time + forecast hour
hrrr = Herbie(
    cycle_dt,
    model="hrrr",
    product="sfc",
    fxx=forecast
)

# RTMA: pass the *valid* analysis time so it matches HRRR's valid time
rtma = Herbie(
    valid_dt,
    model="rtma",
    product="anl"
)


In [ ]:
ds_hrrr = hrrr.xarray("TMP:2 m above")
ds_hrrr

In [ ]:
ds_rtma = rtma.xarray("TMP:2 m above")
ds_rtma

In [ ]:
ds_rtma = ds_rtma.assign_coords(x=ds_rtma.x, y=ds_rtma.y)
ds_hrrr = ds_hrrr.assign_coords(x=ds_hrrr.x, y=ds_hrrr.y)

rtma_on_hrrr = ds_rtma.interp_like(ds_hrrr)
###tempdiff = rtma_on_hrrr.t2m - ds_hrrr.t2m

In [ ]:
# 2) ensure dims align (y,x)
###tempdiff = tempdiff.transpose("y", "x")

In [ ]:
rtma_on_hrrr.sizes["x"] == ds_hrrr.sizes["x"]  # -> 1799

In [ ]:
rtma_on_hrrr.sizes["y"] == ds_hrrr.sizes["y"]  # -> 1059

In [ ]:
rtma_on_hrrr = rtma_on_hrrr.assign_coords(
    latitude=ds_hrrr.latitude,
    longitude=ds_hrrr.longitude,
)

In [ ]:
rtma_on_hrrr["latitude"] == ds_hrrr["latitude"]  # -> True

In [ ]:
rtma_on_hrrr["longitude"] == ds_hrrr["longitude"]  #-> True

In [ ]:
print(ds_hrrr.x.attrs.get("units"), ds_hrrr.y.attrs.get("units"))
print(ds_rtma.herbie.crs)

In [ ]:
# 2) Use HRRR lon/lat + wrap
lon = ds_hrrr["longitude"]
lat = ds_hrrr["latitude"]
lon = xr.where(lon > 180, lon - 360, lon)

# 3) Make a Cartopy map (no EasyMap)
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-95))  # pick your map proj
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
ax.add_feature(cfeature.STATES, linewidth=0.4)
ax.coastlines(resolution="50m", linewidth=0.6)

hrrr_f = (ds_hrrr.t2m - 273.15) * 9/5 + 32

# 4) Plot (coordinates are lon/lat centers)
pc = ccrs.PlateCarree()
p = ax.pcolormesh(
    lon, lat, hrrr_f,
    cmap="coolwarm",
    transform=pc,         # data coords
    shading="nearest",     # treat lon/lat as cell centers
)

cb = plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.02, shrink=0.85)
cb.set_label("HRRR 2m Temperature")
ax.set_title("HRRR 2m Temperature of " + str(date) + " " + str(init_hour) + "Z run at forecast hour " + str(forecast))
plt.show()

In [ ]:
# 2) Use HRRR lon/lat + wrap
lon = ds_rtma["longitude"]
lat = ds_rtma["latitude"]
lon = xr.where(lon > 180, lon - 360, lon)

# 3) Make a Cartopy map (no EasyMap)
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-95))  # pick your map proj
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
ax.add_feature(cfeature.STATES, linewidth=0.4)
ax.coastlines(resolution="50m", linewidth=0.6)

rtma_f = (ds_rtma.t2m - 273.15) * 9/5 + 32

# 4) Plot (coordinates are lon/lat centers)
pc = ccrs.PlateCarree()
p = ax.pcolormesh(
    lon, lat, rtma_f,
    cmap="coolwarm",
    transform=pc,         # data coords
    shading="nearest",    # treat lon/lat as cell centers
)

cb = plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.02, shrink=0.85)
cb.set_label("RTMA 2m Temperature")
ax.set_title("RTMA 2m Temperature at " + str(date) + " " + str(displayhour) + "Z")
plt.show()

In [ ]:
print("HRRR units:", ds_hrrr.t2m.attrs.get("units"))
print("RTMA units:", rtma_on_hrrr.t2m.attrs.get("units"))


In [ ]:
# Compute HRRR - RTMA difference in Fahrenheit
tempdiff_f = (ds_hrrr.t2m - rtma_on_hrrr.t2m) * 9/5 


# 2) Use HRRR lon/lat + wrap
lon = ds_hrrr["longitude"]
lat = ds_hrrr["latitude"]
lon = xr.where(lon > 180, lon - 360, lon)

# 3) Make a Cartopy map (no EasyMap)
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.LambertConformal(central_longitude=-95))  # pick your map proj
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
ax.add_feature(cfeature.STATES, linewidth=0.4)
ax.coastlines(resolution="50m", linewidth=0.6)

# 4) Plot (coordinates are lon/lat centers)
pc = ccrs.PlateCarree()
p = ax.pcolormesh(
    lon, lat, tempdiff_f,
    cmap="coolwarm",
    transform=pc,         # data coords
    shading="nearest",     # treat lon/lat as cell centers
    vmin=-50, vmax=50
)

cb = plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.02, shrink=0.85)
cb.set_label("HRRR − RTMA 2m Temperature (F)")
ax.set_title("HRRR − RTMA: 2m Temperature Difference at " + str(date) + " " + str(displayhour) + "Z")
plt.show()